In [ ]:
# Author: Farid Javadnejad
# Date: 2025-11-11
# Last Update: 2025-11-12
# 
# DESCRIPTION:
# - Script: Converts 360° video frames into Google Street View-style panoramas
# - INPUT_DIR: Source 360° video file (.mp4)
# - OUTPUT_DIR: Extracted and processed panoramic frames
# - Process:
#     - Splits video into frames
#     - Projects equirectangular frames into cube faces
#     - Reconstructs Street View-like perspective images
#     - Uses OpenCV and image transformation techniques for projection
# 
# DISCLAIMER:
# This script was developed with the assistance of AI tools for debugging, reviewing, and testing.


In [1]:

import os, csv, math, subprocess, tempfile
from pathlib import Path
import gpxpy
import gpxpy.gpx
import numpy as np
from datetime import datetime, timezone, timedelta
from pyproj import Transformer
import av

In [ ]:
#!pip install gpxpy
#!pip install pyproj
#!pip install av
#!pip install python-xmp-toolkit 
#!pip pillow 
#!pip install piexif

In [ ]:
# ---------- CONFIG ----------
from pathlib import Path

# Get ROOT as the parent of the notebook folder
ROOT = Path.cwd().parent   # assumes notebook is in ROOT/notebook

VIDEO = ROOT / "data/input/VID_20251111_130157_00_032.mp4"
GPX   = ROOT / "data/input/VID_20251111_130157_00_032.gpx"
OUT_DIR = ROOT / "data/output_utm"

SPACING_M = 5.0                  # image every N meters
TIME_OFFSET_S = 0.0              # adjust if camera clock != GPS time
CRS_EPSG = 32613                 # e.g., 32613 for UTM Zone 13N (Albuquerque, NM)
USE_QUATERNION = False           # True if you want qw,qx,qy,qz; else we'll output heading
WRITE_EXIF = True                # requires exiftool on PATH

# Ensure output directory exists
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Video path: {VIDEO}")
print(f"GPX path:   {GPX}")
print(f"Output dir: {OUT_DIR}")


Video path: c:\Farid\GitHub\video360-to-streetview\data\input\VID_20251111_130157_00_032.mp4
GPX path:   c:\Farid\GitHub\video360-to-streetview\data\input\VID_20251111_130157_00_032.gpx
Output dir: c:\Farid\GitHub\video360-to-streetview\data\output_utm


In [4]:
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6378137.0
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(a))

def bearing_deg(lat1, lon1, lat2, lon2):
    y = math.sin(math.radians(lon2-lon1))*math.cos(math.radians(lat2))
    x = math.cos(math.radians(lat1))*math.sin(math.radians(lat2)) - \
        math.sin(math.radians(lat1))*math.cos(math.radians(lat2))*math.cos(math.radians(lon2-lon1))
    brng = math.degrees(math.atan2(y, x))
    return (brng + 360) % 360

def interp(t0, t1, v0, v1, v):
    # linear interpolation: find t for value v between v0..v1
    if v1 == v0: return t0
    f = (v - v0) / (v1 - v0)
    return t0 + f*(t1 - t0)

def ypr_to_quaternion(yaw_deg, pitch_deg=0.0, roll_deg=0.0):
    # yaw(Z), pitch(Y), roll(X); Generate qw,qx,qy,qz.  Heading ~ yaw around +Z.
    rz = math.radians(yaw_deg)
    ry = math.radians(pitch_deg)
    rx = math.radians(roll_deg)
    cz, sz = math.cos(rz/2), math.sin(rz/2)
    cy, sy = math.cos(ry/2), math.sin(ry/2)
    cx, sx = math.cos(rx/2), math.sin(rx/2)
    qw = cx*cy*cz + sx*sy*sz
    qx = sx*cy*cz - cx*sy*sz
    qy = cx*sy*cz + sx*cy*sz
    qz = cx*cy*sz - sx*sy*cz
    return (qw, qx, qy, qz)

def ts_to_hhmmss_ms(ts):
    # ts in seconds (float) -> "HH:MM:SS.mmm"
    hh = int(ts // 3600); ts -= hh*3600
    mm = int(ts // 60); ts -= mm*60
    ss = int(ts); ms = int(round((ts - ss)*1000))
    return f"{hh:02d}:{mm:02d}:{ss:02d}.{ms:03d}"


In [5]:
# ---------- 1) READ GPX & BUILD DISTANCE/TIME SERIES ----------
with open(GPX, 'r', encoding='utf-8') as f:
    gpx = gpxpy.parse(f)

points = []
for track in gpx.tracks:
    for seg in track.segments:
        for p in seg.points:
            if p.time is None: continue
            points.append((p.time.replace(tzinfo=timezone.utc), p.latitude, p.longitude, p.elevation or 0.0))
if len(points) < 2:
    raise RuntimeError("Not enough GPX points with time.")

# sort by time, compute cumulative distance and bearing
points.sort(key=lambda r: r[0])
cum = [0.0]
brngs = [0.0]
for i in range(1, len(points)):
    _, lat1, lon1, _ = points[i-1]
    _, lat2, lon2, _ = points[i]
    d = haversine_m(lat1, lon1, lat2, lon2)
    cum.append(cum[-1] + d)
    brngs.append(bearing_deg(lat1, lon1, lat2, lon2))

times = [ (t.timestamp()) for (t,_,_,_) in points ]  # epoch seconds UTC
lats  = [ lat for (_,lat,_,_) in points ]
lons  = [ lon for (_,_,lon,_) in points ]
eles  = [ ele for (_,_,_,ele) in points ]


In [6]:
# ---------- 2) CHOOSE TARGET DISTANCES ----------
total = cum[-1]
targets = np.arange(0.0, total, SPACING_M)
if total - targets[-1] > SPACING_M*0.5:
    targets = np.append(targets, total)


In [7]:
# ---------- 3) INTERPOLATE timestamps, lat/lon/ele, bearing for each target ----------
def interp_series(x, xs, ys):
    # given monotonically increasing xs and same-length ys, return y at x
    # find i s.t. xs[i] <= x <= xs[i+1]
    import bisect
    i = bisect.bisect_left(xs, x)
    if i <= 0: return ys[0]
    if i >= len(xs): return ys[-1]
    return ys[i-1] + (ys[i]-ys[i-1]) * ((x - xs[i-1]) / (xs[i]-xs[i-1]))

samples = []
for d in targets:
    t = interp_series(d, cum, times) + TIME_OFFSET_S
    lat = interp_series(d, cum, lats)
    lon = interp_series(d, cum, lons)
    ele = interp_series(d, cum, eles)
    hdg = interp_series(d, cum, brngs)  # degrees, CW from north
    samples.append( (d, t, lat, lon, ele, hdg) )


In [8]:
# ---------- 4) TRANSFORM coordinates to project CRS (required by CSV) ----------
#   CSV x/y/z must match the project coordinate system & units (e.g., meters in your EPSG).
# If your   project uses WGS84 lat/lon, you could keep x=lon,y=lat,z=ellipsoid_height, but EPSG meters are recommended.
transformer = Transformer.from_crs("EPSG:4326", f"EPSG:{CRS_EPSG}", always_xy=True)

xyz_samples = []
for d, t, lat, lon, ele, hdg in samples:
    x, y = transformer.transform(lon, lat)
    z = ele
    xyz_samples.append( (d, t, x, y, z, lat, lon, hdg) )


In [9]:
# ---------- 5) EXTRACT FRAMES with PyAV ----------
# Convert epoch times -> video-relative seconds using GPX start as zero.
# (Assumes TIME_OFFSET_S was used above to align GPX time to video time.)
target_rel_secs = [ (t_epoch - times[0]) for (_, t_epoch, *_rest) in xyz_samples ]

jpg_files = []

# Open container once; reuse across seeks.
container = av.open(VIDEO)
vstream = next((s for s in container.streams if s.type == 'video'), None)
if vstream is None:
    raise RuntimeError("No video stream found in file.")

tb = vstream.time_base               # Fraction (seconds per tick)
stream_start_pts = vstream.start_time if vstream.start_time is not None else 0
stream_start_sec = float(stream_start_pts * tb)  # seconds offset of first frame

def save_frame_at_seconds(container, stream, target_rel_sec, out_path, tol=0.050):
    """
    Seek to a target time (relative-to-video start, seconds) and save the first frame at/after that time.
    tol: tolerance in seconds for matching frame >= target_rel_sec - tol.
    """
    # Seek to a position slightly before target to ensure we decode the correct frame.
    seek_sec = max(stream_start_sec, stream_start_sec + target_rel_sec - 0.5)  # back off 0.5s
    seek_pts = int(seek_sec / stream.time_base)

    container.seek(seek_pts, stream=stream, any_frame=False, backward=True)

    # Decode frames until we reach target time
    for frame in container.decode(stream):
        if frame.pts is None:
            continue
        frame_abs_sec = float(frame.pts * stream.time_base)         # absolute stream time
        frame_rel_sec = frame_abs_sec - stream_start_sec            # relative to video start

        if frame_rel_sec + tol >= target_rel_sec:
            # Convert to PIL Image and save as JPEG
            img = frame.to_image()
            img.save(out_path, "JPEG", quality=92, optimize=True)
            return True
        # Safety: if we overshoot by >2s without finding, bail to avoid long scans
        if frame_rel_sec - target_rel_sec > 2.0:
            break
    return False

for idx, target_rel in enumerate(target_rel_secs):
    out = OUT_DIR / f"pano_{idx:05d}.jpg"
    ok = save_frame_at_seconds(container, vstream, target_rel, out)
    if not ok:
        # Try a second pass with a larger tolerance / different backoff if needed
        ok = save_frame_at_seconds(container, vstream, target_rel, out, tol=0.150)
        if not ok:
            raise RuntimeError(f"Failed to extract frame near {target_rel:.3f}s for {out.name}")
    jpg_files.append(out)

# Optional: close container (PyAV also closes via GC)
container.close()


In [10]:
# ---------- 6) WRITE EXIF GPS + GPano (python-only; no exiftool) ----------
# Writes GPS EXIF via piexif, and GPano XMP via python-xmp-toolkit (if available).
# Cintoo does not require EXIF/XMP for import (CSV positions/orientations drive placement),
# but GPano helps other 360 viewers recognize the images as spherical.
from fractions import Fraction

try:
    import piexif
except ImportError as e:
    raise RuntimeError(
        "piexif is required for Python-only EXIF writing. Install with: pip install piexif"
    ) from e

# GPano XMP is optional; if python-xmp-toolkit is missing, we’ll write GPS EXIF only.
try:
    from libxmp import XMPFiles, XMPMeta
    from libxmp.core import consts as xmpconsts
    HAVE_XMP = True
except Exception:
    HAVE_XMP = False
    print("Note: 'python-xmp-toolkit' not found; GPano XMP will be skipped. Install with: pip install python-xmp-toolkit")

# We need image dimensions for GPano sizing tags
try:
    from PIL import Image
    HAVE_PIL = True
except Exception:
    HAVE_PIL = False
    if HAVE_XMP:
        print("Note: Pillow not available; GPano sizing tags will be omitted. Install with: pip install pillow")

def _to_dms_rational(val: float):
    """
    Convert decimal degrees to EXIF DMS rationals: ((d,1),(m,1),(s_num,s_den)).
    Returns (sign_char, tuple_of_rationals) where sign_char is 'N'/'S' or 'E'/'W' decided by caller.
    """
    sign = 1 if val >= 0 else -1
    v = abs(val)
    deg = int(v)
    min_float = (v - deg) * 60
    minute = int(min_float)
    sec = (min_float - minute) * 60
    def rat(x):
        f = Fraction(x).limit_denominator(1_000_000)
        return (f.numerator, f.denominator)
    dms = (rat(deg), rat(minute), rat(sec))
    return sign, dms

def write_gps_exif_piexif(jpg_path: Path, lat: float, lon: float, alt_m: float, dt_original_str: str):
    """
    Write GPS EXIF + DateTimeOriginal into the JPEG using piexif.
    dt_original_str format: 'YYYY:MM:DD HH:MM:SS'
    """
    jpg_path = Path(jpg_path)
    exif_dict = {}
    try:
        # load existing EXIF if present (keeps other tags intact)
        exif_dict = piexif.load(str(jpg_path))
    except Exception:
        exif_dict = {"0th": {}, "Exif": {}, "GPS": {}, "1st": {}, "thumbnail": None}

    lat_sign, lat_dms = _to_dms_rational(lat)
    lon_sign, lon_dms = _to_dms_rational(lon)

    gps_ifd = exif_dict.get("GPS", {})
    gps_ifd[piexif.GPSIFD.GPSVersionID] = (2, 0, 0, 0)
    gps_ifd[piexif.GPSIFD.GPSLatitudeRef]  = 'N' if lat_sign >= 0 else 'S'
    gps_ifd[piexif.GPSIFD.GPSLatitude]     = lat_dms
    gps_ifd[piexif.GPSIFD.GPSLongitudeRef] = 'E' if lon_sign >= 0 else 'W'
    gps_ifd[piexif.GPSIFD.GPSLongitude]    = lon_dms
    gps_ifd[piexif.GPSIFD.GPSAltitudeRef]  = 0 if alt_m >= 0 else 1
    # Altitude is a rational; use centimeters denominator to preserve decimals
    gps_ifd[piexif.GPSIFD.GPSAltitude]     = (int(round(abs(alt_m) * 100)), 100)

    exif_ifd = exif_dict.get("Exif", {})
    exif_ifd[piexif.ExifIFD.DateTimeOriginal] = dt_original_str

    exif_dict["GPS"]  = gps_ifd
    exif_dict["Exif"] = exif_ifd

    exif_bytes = piexif.dump(exif_dict)
    piexif.insert(exif_bytes, str(jpg_path))

def write_gpano_xmp(jpg_path: Path, heading_deg: float):
    """
    Write GPano XMP tags (ProjectionType, UsePanoramaViewer, PoseHeadingDegrees, sizing).
    Requires python-xmp-toolkit; silently no-op if not available.
    """
    if not HAVE_XMP:
        return

    jpg_path = Path(jpg_path)
    ns = "http://ns.google.com/photos/1.0/panorama/"
    prefix = "GPano"

    # Optional: set sizing tags using actual image width/height when Pillow is available
    width = height = None
    if HAVE_PIL:
        try:
            with Image.open(jpg_path) as im:
                width, height = im.size
        except Exception:
            pass

    xmpfile = XMPFiles(file_path=str(jpg_path), open_forupdate=True)
    try:
        xmp = xmpfile.get_xmp() or XMPMeta()
        if not xmp.does_namespace_exist(ns):
            XMPMeta.register_namespace(ns, prefix)

        # Required/important GPano tags for spherical 360:
        xmp.set_property(ns, "ProjectionType", "equirectangular")
        xmp.set_property(ns, "UsePanoramaViewer", True)
        # Heading in degrees clockwise from North
        xmp.set_property(ns, "PoseHeadingDegrees", f"{float(heading_deg):.2f}")

        # Helpful sizing tags so viewers render correctly without guessing
        if width and height:
            xmp.set_property(ns, "CroppedAreaImageWidthPixels",  str(width))
            xmp.set_property(ns, "CroppedAreaImageHeightPixels", str(height))
            xmp.set_property(ns, "FullPanoWidthPixels",          str(width))
            xmp.set_property(ns, "FullPanoHeightPixels",         str(height))
            xmp.set_property(ns, "CroppedAreaLeftPixels",        "0")
            xmp.set_property(ns, "CroppedAreaTopPixels",         "0")

        xmpfile.put_xmp(xmp)
    finally:
        xmpfile.close_file()

if WRITE_EXIF:
    for idx, jf in enumerate(jpg_files):
        _, t_epoch, x, y, z, lat, lon, hdg = xyz_samples[idx]
        ts = datetime.fromtimestamp(t_epoch, tz=timezone.utc).strftime("%Y:%m:%d %H:%M:%S")
        # 1) GPS EXIF (always, via piexif)
        write_gps_exif_piexif(jf, lat, lon, z, ts)
        # 2) GPano XMP (optional, via python-xmp-toolkit)
        write_gpano_xmp(jf, hdg)

Note: 'python-xmp-toolkit' not found; GPano XMP will be skipped. Install with: pip install python-xmp-toolkit


In [11]:
# ---------- 7) BUILD   CSV ----------
csv_path = OUT_DIR / "panos.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    if USE_QUATERNION:
        writer = csv.writer(f)
        writer.writerow(["filename","x","y","z","qw","qx","qy","qz"])
        for idx, jf in enumerate(jpg_files):
            _, _, x, y, z, _lat, _lon, hdg = xyz_samples[idx]
            qw,qx,qy,qz = ypr_to_quaternion(yaw_deg=hdg, pitch_deg=0.0, roll_deg=0.0)
            writer.writerow([jf.name, f"{x:.3f}", f"{y:.3f}", f"{z:.3f}", f"{qw:.8f}", f"{qx:.8f}", f"{qy:.8f}", f"{qz:.8f}"])
    else:
        writer = csv.writer(f)
        writer.writerow(["filename","x","y","z","heading"])
        for idx, jf in enumerate(jpg_files):
            _, _, x, y, z, _lat, _lon, hdg = xyz_samples[idx]
            writer.writerow([jf.name, f"{x:.3f}", f"{y:.3f}", f"{z:.3f}", f"{hdg:.2f}"])

In [12]:
print(f"Done. Panos in: {OUT_DIR}")
print(f"CSV for Panos: {csv_path}")

Done. Panos in: c:\Farid\GitHub\video360-to-streetview\data\output_utm
CSV for Panos: c:\Farid\GitHub\video360-to-streetview\data\output_utm\panos.csv


In [13]:
# ---------- A) Viewer choice & assets in {ROOT}\tools ----------
from pathlib import Path
import urllib.request

# Choose the viewer engine: 'pannellum' (simplest) or 'marzipano'
VIEWER = 'pannellum'   # change to 'marzipano' if you prefer

TOOLS = Path(ROOT) / "tools"
TOOLS.mkdir(parents=True, exist_ok=True)

def ensure_pannellum_in_tools():
    """
    Ensures Pannellum assets exist at:
      {ROOT}\tools\pannellum\pannellum.js
      {ROOT}\tools\pannellum\pannellum.css
    Downloads them if they are missing (requires internet), otherwise
    instructs to stage them manually.
    """
    base = TOOLS / "pannellum"
    js  = base / "pannellum.js"
    css = base / "pannellum.css"
    base.mkdir(parents=True, exist_ok=True)
    if not js.exists():
        try:
            urllib.request.urlretrieve(
                "https://cdn.jsdelivr.net/npm/pannellum@2.5.6/build/pannellum.js",
                js.as_posix()
            )
        except Exception as e:
            raise RuntimeError(f"Download failed. Please place pannellum.js at: {js}") from e
    if not css.exists():
        try:
            urllib.request.urlretrieve(
                "https://cdn.jsdelivr.net/npm/pannellum@2.5.6/build/pannellum.css",
                css.as_posix()
            )
        except Exception as e:
            raise RuntimeError(f"Download failed. Please place pannellum.css at: {css}") from e
    return {"base": base, "js": js, "css": css}

def ensure_marzipano_in_tools():
    """
    Ensures Marzipano asset exists at:
      {ROOT}\tools\marzipano\marzipano.min.js
    Downloads a compatible build if missing.
    """
    base = TOOLS / "marzipano"
    js = base / "marzipano.min.js"
    base.mkdir(parents=True, exist_ok=True)
    if not js.exists():
        try:
            # A lightweight build compatible with equirect panos
            urllib.request.urlretrieve(
                "https://www.marzipano.net/demos/marzipano.js",
                js.as_posix()
            )
        except Exception as e:
            raise RuntimeError(f"Download failed. Please place marzipano.min.js at: {js}") from e
    return {"base": base, "js": js}

if VIEWER == 'pannellum':
    VIEWER_FILES = ensure_pannellum_in_tools()
elif VIEWER == 'marzipano':
    VIEWER_FILES = ensure_marzipano_in_tools()
else:
    raise ValueError("VIEWER must be 'pannellum' or 'marzipano'")

print("Viewer assets staged in:", VIEWER_FILES["base"])

Viewer assets staged in: c:\Farid\GitHub\video360-to-streetview\tools\pannellum


<>:14: SyntaxWarning: invalid escape sequence '\p'
<>:44: SyntaxWarning: invalid escape sequence '\m'
<>:14: SyntaxWarning: invalid escape sequence '\p'
<>:44: SyntaxWarning: invalid escape sequence '\m'
C:\Users\USFJ139860\AppData\Local\Temp\ipykernel_48036\3951276024.py:14: SyntaxWarning: invalid escape sequence '\p'
  {ROOT}\tools\pannellum\pannellum.js
C:\Users\USFJ139860\AppData\Local\Temp\ipykernel_48036\3951276024.py:44: SyntaxWarning: invalid escape sequence '\m'
  {ROOT}\tools\marzipano\marzipano.min.js


In [14]:
# ---------- B) Create viewer.html (references assets in {ROOT}\tools) ----------
from pathlib import Path
import os

viewer_html = OUT_DIR / "viewer.html"

def relpath_from_outdir(target: Path) -> str:
    """Return a forward-slash relative path from OUT_DIR to target."""
    return Path(os.path.relpath(target, start=OUT_DIR)).as_posix()

if VIEWER == 'pannellum':
    pann_js  = relpath_from_outdir(VIEWER_FILES["js"])
    pann_css = relpath_from_outdir(VIEWER_FILES["css"])
    html = f"""<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>360 Viewer (Pannellum)</title>
  <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <link rel="stylesheet" href="{pann_css}">
  <style>
    html,body,#panocontainer {{height:100%;width:100%;margin:0;padding:0;background:#000;}}
  </style>
</head>
<body>
  <div id="panocontainer"></div>
  <script>
    function getParams() {{
      const qs = new URLSearchParams(window.location.search);
      return {{
        img: qs.get('img') || '',
        yaw: parseFloat(qs.get('yaw') || '0')   // degrees
      }};
    }}
  </script>
  <script src="{pann_js}"></script>
  <script>
    const p = getParams();
    if(!p.img) {{
      document.body.innerHTML = "<p style='font:16px sans-serif;color:#fff;padding:20px'>Missing ?img=FILENAME query.</p>";
    }} else {{
      pannellum.viewer('panocontainer', {{
        type: 'equirectangular',
        panorama: p.img,       // relative to viewer.html (OUT_DIR)
        autoLoad: true,
        compass: true,
        yaw: isNaN(p.yaw) ? 0 : p.yaw,   // degrees
        hfov: 100
      }});
    }}
  </script>
</body>
</html>"""
    viewer_html.write_text(html, encoding="utf-8")

elif VIEWER == 'marzipano':
    marzi_js = relpath_from_outdir(VIEWER_FILES["js"])
    html = f"""<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>360 Viewer (Marzipano)</title>
  <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <style>
    html,body,#pano {{height:100%;width:100%;margin:0;padding:0;background:#000;}}
  </style>
</head>
<body>
  <div id="pano"></div>
  <script>
    function getParams(){{
      const qs = new URLSearchParams(window.location.search);
      return {{ img: qs.get('img') || '', yaw: parseFloat(qs.get('yaw') || '0') }};
    }}
  </script>
  <script src="{marzi_js}"></script>
  <script>
    const p = getParams();
    if(!p.img){{
      document.body.innerHTML = "<p style='font:16px sans-serif;color:#fff;padding:20px'>Missing ?img=FILENAME query.</p>";
    }} else {{
      var viewer = new Marzipano.Viewer(document.getElementById('pano'));
      var img = new Image();
      img.onload = function(){{
        var geometry = new Marzipano.EquirectGeometry([{{ width: img.naturalWidth }}]);
        var limiter = Marzipano.RectilinearView.limit.traditional(1024, 120 * Math.PI / 180);
        var view = new Marzipano.RectilinearView({{ yaw: (p.yaw||0) * Math.PI/180 }}, limiter);
        var source = Marzipano.ImageUrlSource.fromString(p.img);
        var scene = viewer.createScene({{ source: source, geometry: geometry, view: view }});
        scene.switchTo();
      }};
      img.src = p.img;
    }}
  </script>
</body>
</html>"""
    viewer_html.write_text(html, encoding="utf-8")

print("viewer.html written:", viewer_html)


viewer.html written: c:\Farid\GitHub\video360-to-streetview\data\output_utm\viewer.html


In [15]:
# ---------- C) Write KML (thumbnail + interactive viewer link) ----------
from urllib.parse import quote

KML_PATH = OUT_DIR / "panos.kml"
thumb_width = 300       # px
USE_HTTP_LINKS = False  # set True if you run Cell D (local web server)
YAW_SIGN = 1.0          # set to -1.0 if the viewer faces the opposite way

if USE_HTTP_LINKS:
    # viewer.html must be accessible at http://127.0.0.1:8000/viewer.html
    base_href = "http://127.0.0.1:8000/viewer.html?img="
else:
    # viewer.html is in OUT_DIR and opened via file://, so a relative link works
    base_href = "viewer.html?img="

with open(KML_PATH, "w", encoding="utf-8") as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    f.write('<kml xmlns="http://www.opengis.net/kml/2.2">\n')
    f.write('  <Document>\n')
    f.write('    <name>360 Panos</name>\n')

    for idx, jf in enumerate(jpg_files):
        # xyz_samples[idx] = (d, t, x, y, z, lat, lon, hdg)
        _, _, x, y, z, lat, lon, hdg = xyz_samples[idx]
        photo_id = jf.name
        coords = f"{lon:.8f},{lat:.8f},{z:.2f}"

        # URL-encode filename for the query string
        q_img = quote(photo_id)
        q_yaw = f"{YAW_SIGN * hdg:.2f}"

        if USE_HTTP_LINKS:
            thumb_src = f"http://127.0.0.1:8000/{quote(photo_id)}"
            link_href = f"{base_href}{q_img}&yaw={q_yaw}"
        else:
            thumb_src = photo_id
            link_href = f"{base_href}{q_img}&yaw={q_yaw}"

        desc = f"""<![CDATA[
<p><b>{photo_id}</b></p>
<p>Heading: {hdg:.2f}°</p>
<p><img src="{thumb_src}" width="{thumb_width}"></p>
<p><a href="{link_href}">Open Full Image (Interactive)</a></p>
]]>"""

        f.write('    <Placemark>\n')
        f.write(f'      <name>{photo_id}</name>\n')
        f.write(f'      <description>{desc}</description>\n')
        f.write('      <Point>\n')
        f.write(f'        <coordinates>{coords}</coordinates>\n')
        f.write('      </Point>\n')
        f.write('    </Placemark>\n')

    f.write('  </Document>\n')
    f.write('</kml>\n')

print("KML written:", KML_PATH)

KML written: c:\Farid\GitHub\video360-to-streetview\data\output_utm\panos.kml
